<a href="https://colab.research.google.com/github/seoho0529/TensorFlow/blob/main/tf46Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformer : RNN을 제거하고 Attention으로만 구성된 서로 다른 인코더와 디코더가 여러 개 쌓인 형태의 네트워크.
# 인코더 또는 디코더 안에는 self attention과 feedford 신경망(keras : Dense)으로 구성 되어 있다.
# self attention 함수는 주어진 Query에 대해서 key와 유사도를 구한다. 그리고 이 유사도를 키와 매핑되어 있는 값(value)에 반영한다.
# self attention은 RNN 처럼 순서대로 처리 하는 방식이 아니라 해당하는 단어와 관련된 뜻을 찾기 위한 어텐션을 말한다.

# 간단한 구조 이해
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential


In [2]:
# load data
vocab_size = 20000
maxlen=200

(x_train,y_train),(x_val,y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), len(x_val))

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)
print(x_train[2])

17464789/17464789 [==============================] - 0s 0us/step
25000 25000
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    1   14   47    8   30   31    7    4  249  108    7
    4 5974   54   61  369   13   71  149   14   22  112    4 2401  311
   12   16 3711   33   75   43 1829  296    4   86  320   35  534   19
  263 4821 1301    4 1873   33   89   78   12   66   16    4  360    7
    4   58  316  334   11    4 1716   43  645  662    8  257   85 1200
   42 1228 2578   83   68 3912   15   36  165 1539  278   36   69    2
  780    8  106   14 6905 1338   18    6   22   12  215   28  610   40
    6   87  326   23 2300   21   23   22   12  272   40   57   31   11
    4   22   47    6 2307   51    9  170   23  595  116  595 1352   13


In [3]:
# Token and Position embedding
inputs = layers.Input(shape=(200,))
x = layers.Embedding(input_dim=128, output_dim=32)(inputs)
print(tf.keras.Model(inputs=inputs, outputs=x).summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 32)           4096      
                                                                 
Total params: 4096 (16.00 KB)
Trainable params: 4096 (16.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [10]:
inputs = layers.Input(shape=(200,))
x = layers.Embedding(input_dim=128, output_dim=32)(inputs)
positions = tf.range(start=0, limit=200, delta=1)
# 각 단어 위치에 따른 index값을 embedding하여 feature를 추출 한 후, 합을 구함(단어embedding 결과 + 위치embedding결과)
positions = layers.Embedding(input_dim=200, output_dim=32)(positions)
embedding_layer = x + positions

# Multi-head attention을 위한 간단한 구조를 작성
EmbeddingDim=5
WordLen=5

inputs = layers.Input(shape=(WordLen))
positions = tf.range(start=0, limit=WordLen, delta=1)
positions = layers.Embedding(input_dim=WordLen, output_dim=EmbeddingDim)(positions)
x = layers.Embedding(input_dim=200, output_dim=EmbeddingDim)(inputs)
embedding_layer = x + positions

# num_heads=1 : 현재 Attention Matrix를 만드는 작업을 몇 번할 것인가를 결정
attention_output = layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=False)(embedding_layer, embedding_layer)
print(tf.keras.Model(inputs=inputs, outputs=attention_output).summary())


model = tf.keras.Model(inputs=inputs, outputs=attention_output)
print([w.shape for w in model.get_weights()])
# [(200, 5),               (5, 1, 1), (5, 1, 1),        (5, 1, 1),             (1, 1, 5)]
# embedding_layer weight,     Query,      Key,      Value 에 대한 weight      dot production 내부에서 transpose되어 순서가 변경

print('----------------------------------------------')
# MultiHeadAttention layer에 num_heads=2, key_dim=3 으로 변경해보자
inputs = layers.Input(shape=(WordLen,))
positions = tf.range(start=0, limit=WordLen, delta=1)
positions = layers.Embedding(input_dim=WordLen, output_dim=EmbeddingDim)(positions)
x = layers.Embedding(input_dim=200, output_dim=EmbeddingDim)(inputs)
embedding_layer = x + positions

# num_heads=1 : 현재 Attention Matrix를 만드는 작업을 몇 번할 것인가를 결정
# attention_output = layers.MultiHeadAttention(num_heads=2, key_dim=3, use_bias=False)(embedding_layer, embedding_layer)
attention_output = layers.MultiHeadAttention(num_heads=2, key_dim=3, use_bias=True)(embedding_layer, embedding_layer)
print(tf.keras.Model(inputs=inputs, outputs=attention_output).summary())


model = tf.keras.Model(inputs=inputs, outputs=attention_output)
print([w.shape for w in model.get_weights()])
# [(200, 5), (5, 2, 3), (5, 2, 3), (5, 2, 3), (2, 3, 5)]
# [(200, 5), (5, 2, 3), (2, 3), (5, 2, 3), (2, 3), (5, 2, 3), (2, 3), (2, 3, 5), (5,)]  use_bias=True하면 중간에 상수가 더해짐
# 파라미터에 대한 존재 이유, 연산 등의 이해가 있다면, 다음으로 Scaled dot production Attention만 이해하면 Transformer에 기초 끝!!

# 단어들의 유사도와 같은 역할을 하는 Attention score를 구하고 이를 Value에 곱해주는 작업을 Single-Head Attention이 한다.
# Single-Head Attention을 여러개 병렬로 처리하면 이게 바로 Multi - Head Attention이 된다.
# 이 때, num_heads는 이 Attention matrix를 만드는 수행을 몇 번 할 것인가를 의미함.

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, 5)]                  0         []                            
                                                                                                  
 embedding_32 (Embedding)    (None, 5, 5)                 1000      ['input_17[0][0]']            
                                                                                                  
 tf.__operators__.add_15 (T  (None, 5, 5)                 0         ['embedding_32[0][0]']        
 FOpLambda)                                                                                       
                                                                                                  
 multi_head_attention_8 (Mu  (None, 5, 5)                 20        ['tf.__operators__.add_

In [12]:
# 이제 MultiHeadAttention을 통과한 분류 모델을 작성
EmbeddingDim = 128
WordLen = 200

inputs = layers.Input(shape=(WordLen,))
positions = tf.range(start=0, limit=WordLen, delta=1)
positions = layers.Embedding(input_dim=WordLen, output_dim=EmbeddingDim)(positions)
x = layers.Embedding(input_dim=200, output_dim=EmbeddingDim)(inputs)
embedding_layer = x + positions


attention_output = layers.MultiHeadAttention(num_heads=1, key_dim=2, use_bias=True)(embedding_layer, embedding_layer)

x = layers.GlobalAvgPool1D()(attention_output)
# MultiHeadAttention을 통과한 output은 Embedding의 output과 동일하므로 GlobalAveragePooling1D을 통해 Vector화 할 수 있다.
outputs = layers.Dense(2, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=attention_output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())
print([w.shape for w in model.get_weights()])

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 200)]                0         []                            
                                                                                                  
 embedding_38 (Embedding)    (None, 200, 128)             25600     ['input_20[0][0]']            
                                                                                                  
 tf.__operators__.add_18 (T  (None, 200, 128)             0         ['embedding_38[0][0]']        
 FOpLambda)                                                                                       
                                                                                                  
 multi_head_attention_11 (M  (None, 200, 128)             1158      ['tf.__operators__.add_